In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision 
import pandas as pd

In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\covid.train.csv',)
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\covid.test.csv')

In [3]:
train_data.iloc[:5,[0,1,2,3,-3,-2,-1]]

,id,AL,AK,AZ,worried_become_ill.2,worried_finances.2,tested_positive.2
0,0,1.0,0.0,0.0,53.991549,43.604229,20.704935
1,1,1.0,0.0,0.0,54.185521,42.665766,21.292911
2,2,1.0,0.0,0.0,53.637069,42.972417,21.166656
3,3,1.0,0.0,0.0,52.446223,42.907472,19.896607
4,4,1.0,0.0,0.0,52.560315,43.321985,20.178428


In [4]:
train_data.shape

(2700, 95)

In [5]:
all_features=pd.concat((train_data.iloc[:,1:-1],test_data.iloc[:,1:]))

In [6]:
all_features.shape

(3593, 93)

transformer to Norm(0,1)

In [7]:
all_features.iloc[:,40:]=all_features.iloc[:,40:].apply(
lambda x: (x-x.mean())/ (x.std()))

In [8]:
n_train=train_data.shape[0]
train_features=torch.Tensor(all_features.iloc[:n_train,:].values)
test_features=torch.Tensor(all_features.iloc[n_train:,:].values)
train_labels=torch.Tensor(train_data.iloc[:,-1].values).view(-1,1)

define net and func

In [9]:
loss=nn.MSELoss()

In [10]:
def rmse(net,features,labels):
    with torch.no_grad():
        rmse=torch.sqrt(2*loss(net(features),labels).mean())
    return rmse.item()

In [11]:
def get_net(num_features):
    net=nn.Sequential(
        nn.Linear(num_features,256),
        nn.ReLU(),
        nn.Linear(256,1)
    )
    return net

In [12]:
def train(net,train_features,train_labels,test_features,test_labels,num_epochs,
         learning_rate,weight_decay,batch_size):
    train_ls,test_ls=[],[]
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    train_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    optimizer=torch.optim.Adam(net.parameters(),lr=learning_rate,
                               weight_decay=weight_decay)
    for e in range(num_epochs):
        for x,y in train_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
        train_ls.append(rmse(net,train_features,train_labels))
        if test_labels is not None:
            test_ls.append(rmse(net,test_features,test_labels))
    return train_ls,test_ls
    

In [13]:
def get_k_fold_data(k,i,x,y):
    assert k>1
    fold_size=x.shape[0]//k
    x_train,y_train=None,None
    for j in range(k):
        idx=slice(j*fold_size,(j+1)*fold_size)
        x_part,y_part=x[idx,:],y[idx]
        if j==i:
            x_valid,y_valid=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_valid,y_valid

In [14]:
def k_fold(k,x_train,y_train,num_epochs,lr,weight_decay,batch_size):
    train_l_sum,valid_l_sum=0,0
    for i in range(k):
        data=get_k_fold_data(k,i,x_train,y_train)
        net=get_net(x_train.shape[1])
        train_ls,valid_ls=train(net,*data,num_epochs,lr,weight_decay,batch_size)
        train_l_sum+=train_ls[-1]
        valid_l_sum+=valid_ls[-1]
        print('fold %d ,train rmse %f,valid rmse %f ' % (
        i,train_ls[-1],valid_ls[-1]))
    return train_l_sum/k,valid_l_sum/k

In [17]:
k,num_epochs,lr,weight_decay,batch_size=5,100,0.05,0,64

In [18]:
train_l,valid_l=k_fold(k,train_features,train_labels,num_epochs,lr,
                      weight_decay,batch_size)
print('%d-fold validation : avg train rmse %f,avg valid rmse %f' % (
k,train_l,valid_l))

fold 0 ,train rmse 1.186547,valid rmse 3.828812 
fold 1 ,train rmse 1.281961,valid rmse 7.523718 
fold 2 ,train rmse 1.180311,valid rmse 5.327919 
fold 3 ,train rmse 1.401944,valid rmse 6.241438 
fold 4 ,train rmse 1.500206,valid rmse 5.292349 
5-fold validation : avg train rmse 1.310194,avg valid rmse 5.642847
